## Data cleaning and problem solving

Import needed libraries

In [1]:
import pandas as pd
import numpy as np
import re

Uploading data

In [2]:
df_raw = pd.read_csv(r"C:\Users\wh\Desktop\us_tax_data\dateset.xlsx - consumption_data.csv")
df_tax_sel = pd.read_csv(r"C:\Users\wh\Desktop\us_tax_data\dateset.xlsx - consumption_data.csv")
df_cons = pd.read_csv(r"C:\Users\wh\Desktop\us_tax_data\dateset.xlsx - consumption_data.csv")


#### Cleaning

In [ ]:
### Cleaning df_raw
# dropping all completely empty raws
df_raw_2 = df_raw.dropna(how='all')

# resetting the index
df_raw_2 = df_raw_2.reset_index(drop='True')
df_raw_2.head(10)

# getting rid of regions in State/Region column
df_raw_2 = df_raw_2[df_raw_2["State/Region"].str.contains("Region") == False]
df_raw_2 = df_raw_2.reset_index(drop='True')
df_raw_2.head(10)

# left joining df_raw and df_tax_sel
merged = df_raw_2.merge(df_tax_sel, on='Tax Code', how='left', indicator=True)

# deleting all rows which contain 'no' in include
merged2 = merged[merged['Include?'] != 'no']
merged2 = merged2.reset_index(drop='True')

# checking whether there still are rows with 'Include?' == 'no'
merged2[merged2['Include?']=='no'].count()
merged2.head(10)

# making population readable as int
merged2['Population'] = merged2['Population'].str.extract('(\d+)', expand=False)
merged2['Population'] = merged2['Population'].replace(np.nan, '0', regex=True)
merged2['Population'] = merged2['Population'].astype(int)
merged2['Population'] = merged2['Population'].replace(0, np.nan, regex=True)
merged2

# removing all special characters in tax names
merged2['Tax Name'] = merged2['Tax Name'].str.replace('_N/A_', '')
merged2
merged2['Tax Name'] = merged2['Tax Name'].str.replace('\W', '')
merged2['Tax Name'] = merged2['Tax Name'].str.replace('_', '')
merged2['Tax Name'].unique()

# adding whitespace after each capital letter
tax_names = merged2['Tax Name']
tax_names = tax_names.replace(np.nan, 'Empty', regex=True)
for i in range(len(tax_names)):
    tax_names[i] = re.sub(r"(\w)([A-Z])", r"\1 \2", tax_names[i])
tax_names = tax_names.replace('Empty', np.nan, regex=True)
tax_names.unique()
merged2['Tax Name 2'] = tax_names
# Still one name is wrong, there is a whitespace missing
merged2['Tax Name 2'] = merged2['Tax Name 2'].str.replace('Corporationsin', 'Corporations in')
merged2['Tax Name 2'].unique()

# checking if the number of included taxes is correct
print(df_tax_sel[df_tax_sel['Include?']=='yes'].count() == merged2['Tax Name 2'].nunique())

# checking if the columns are in correct format
merged2.dtypes

# taxes collected needs to be int
merged2['Taxes Collected'].unique()

# 'X' needs to be removed
merged2['Taxes Collected'] = merged2['Taxes Collected'].replace('X', '0', regex=True)
merged2['Taxes Collected'] = merged2['Taxes Collected'].replace(np.nan, '0', regex=True)
merged2['Taxes Collected'] = merged2['Taxes Collected'].astype(int)
merged2['Taxes Collected'] = merged2['Taxes Collected'].replace(0, np.nan, regex=True)

# checking the column types again
merged2.dtypes

### cleaning consumption data 
# taking only rows and columns that are needed
df_cons
df_cons2 = df_cons.iloc[1:, 1:]

# setting first row as column names
df_cons2 = df_cons2.rename(columns=df_cons2.iloc[0])
df_cons2 = df_cons2.reset_index(drop = 'True')

# getting rid of first row which is a duplicate of the column names
df_cons2 = df_cons2.iloc[1:4]
df_cons2 = df_cons2.reset_index()

# deleting index column 
del df_cons2['index']

# transposing data to join it after with merged2 df
df_cons_t = df_cons2.transpose()

# setting first row as column names
df_cons_t = df_cons_t.rename(columns=df_cons_t.iloc[0])

# Taking states out from index and adding as a seperate column
df_cons_t['State'] = df_cons_t.index

# taking everything except of the first row
df_cons_t = df_cons_t.iloc[1:,:]
df_cons_t = df_cons_t.reset_index(drop='True')
df_cons_t['State'] = df_cons_t['State'].replace('United States total', 'United States', regex=True)
df_cons_t['2018'] = df_cons_t['2018'].astype(int)
df_cons_t['2019'] = df_cons_t['2019'].astype(int)
df_cons_t['2020'] = df_cons_t['2020'].astype(int)
df_cons_t.dtypes
df_cons_t

# correcting district of columbia to dc
df_cons_t['State'] = df_cons_t['State'].replace('District of Columbia', 'DC', regex=True)

# joining df_cons_t with merged2
merged_all = merged2.merge(df_cons_t, left_on='State/Region',right_on ='State' ,how='left')
merged_all = merged_all[merged_all["State/Region"].str.contains("United States") == False]
merged_all.head(10)

#creating a final df with columns needed to answer the questions and getting rid of United States in total
final = merged_all[['State/Region', 'Taxes Collected', 'Population', 'Tax Name 2', 'Region']]
final



#### Problem solving

In [6]:
# 1.Determine state which has the highest amount of taxes per person.
sum_taxes = final.groupby(by='State/Region')['Taxes Collected'].sum()
pop = final.groupby(by='State/Region')['Population'].max()
taxes_per_person = sum_taxes/pop
print("Solution for Problem 1")
print(taxes_per_person.sort_values(ascending=False).head(1))

# 2.Determine state with the third largest amount of collected taxes.
taxes_collected = final.groupby(by='State/Region')['Taxes Collected'].sum().sort_values(ascending=False)
print("Solution for Problem 2")
print(taxes_collected.head(3))

# 3.Name region with the smallest value of sum of "Individual Income Taxes" and "Income Taxes".
only_two_taxes = final[final['Tax Name 2'].isin(['Individual Income Taxes','Income Taxes'])]
only_two_taxes_s = only_two_taxes.groupby(by='Region')['Taxes Collected'].sum().sort_values(ascending=True)
print("Solution for Problem 3")
print(only_two_taxes_s.head(1))

# 4.Calculate percent changes of personal consumption expenditures between 2018 and 2020 for each state and determine state and year with highest change (example: Florida 2018/2019)
final_4 = merged_all[['State/Region', '2018', '2019', '2020']]
final_5 = final_4.drop_duplicates()
final_5 = final_5.reset_index(drop='True')
# calculating the percentage change 
final_5['Change 2018/2019'] = round(((final_5['2019']-final_5['2018'])/final_5['2018']*100),2)
final_5['Change 2019/2020'] = round(((final_5['2020']-final_5['2019'])/final_5['2019']*100),2)
final_5 = final_5[['State/Region', 'Change 2018/2019', 'Change 2019/2020']]
final_5[['State/Region', 'Change 2018/2019']].groupby(by='Change 2018/2019').max().sort_values(by = 'Change 2018/2019',ascending=False).head(1)
print("Solution for Problem 4")
print(final_5[['State/Region', 'Change 2019/2020']].groupby(by='Change 2019/2020').max().sort_values(by = 'Change 2019/2020',ascending=False).head(1))


# 5.Determine Region with the highest average Personal Consumption Expenditures per person in 2020
# keep in mind to multiply by million (million of dollars consumption)
final_6 = merged_all[['Region','2020', 'Population']]
final_6 = final_6.dropna()
final_6 = final_6.reset_index(drop='True')
final_6 = final_6.groupby(by='Region').sum()
final_6['avg_per_con'] = (final_6['2020']/final_6['Population'])
print("Solution for Problem 5")
print(final_6.sort_values(by = 'avg_per_con',ascending=False))

Solution for Problem 1
State/Region
DC    21.914669
dtype: float64
Solution for Problem 2
State/Region
California    347066701.0
New York      177020522.0
Illinois       70675224.0
Name: Taxes Collected, dtype: float64
Solution for Problem 3
Region
Western South Region    18526820.0
Name: Taxes Collected, dtype: float64
Solution for Problem 4
                 State/Region
Change 2019/2020             
1.24                    Idaho
Solution for Problem 5
                             2020  Population  avg_per_con
Region                                                    
New England Region         744663  14802967.0     0.050305
Middle Atlantic Region    1978557  41269709.0     0.047942
Pacific Region            2467638  53135362.0     0.046441
Southern Atlantic Region  2752023  64641801.0     0.042573
Mountain Region           1011715  24184624.0     0.041833
Western Midwest Region     880031  21277130.0     0.041360
Eastern Midwest Region    1927402  46878905.0     0.041114
Western Sou

#### Exporting Data for Google Data Studio to visualise it 

In [ ]:
merged_all.to_csv(r"C:\Users\wh\Desktop\us_tax_data\merged_all.csv")

df_cons_t_2=  df_cons_t.merge(merged2, left_on='State',right_on ='State/Region' ,how='left')
df_cons_t_2 = df_cons_t_2[['2018', '2019', '2020', 'State', 'Region', 'Population']]
df_cons_t_2 = df_cons_t_2.dropna()
df_cons_t_2.to_csv(r"C:\Users\wh\Desktop\us_tax_data\consumption_over_time_2.csv")

#### You can view the Dashboad via the link below

https://lookerstudio.google.com/reporting/e7beec96-ea03-4353-a26d-304feb9bbee8

This Dashboad wass built within 3 hours, therefore the Visualization could be done pettie, but I focused on visualising the insights